# Get data

In [2]:
!git clone https://github.com/khumbuai/ulmfit_keras.git

Cloning into 'ulmfit_keras'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 536 (delta 0), reused 0 (delta 0), pack-reused 535
Receiving objects: 100% (536/536), 108.75 KiB | 6.80 MiB/s, done.
Resolving deltas: 100% (343/343), done.


In [0]:
import os
os.chdir('./ulmfit_keras/')

In [4]:
!chmod -R 777 *
!./get_data.sh

=== Acquiring datasets ===
---
- Downloading WikiText-2 (WT2)
- Downloading WikiText-103 (WT2)
--2019-05-10 13:07:35--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: ‘wikitext-103-v1.zip’

wikitext-103-v1.zip 100%[===================>] 181.42M  45.8MB/s    in 4.4s    

2019-05-10 13:07:40 (41.5 MB/s) - ‘wikitext-103-v1.zip’ saved [190229076/190229076]

- Downloading enwik8 (Character)
--2019-05-10 13:07:45--  http://mattmahoney.net/dc/enwik8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36445475 (35M) [application/zip]
Saving to: ‘enwik8.zip’

enwik8.zip        

In [0]:
!mkdir keras_lm/assets

In [0]:
!cp -rf data/* keras_lm/assets

In [0]:
!rm -rf data

In [0]:
import os
os.chdir('./keras_lm')



## Datasets

## Pretrained models

In [9]:

!wget -P assets/wikitext-103 http://files.fast.ai/models/wt103/itos_wt103.pkl

--2019-05-10 13:08:54--  http://files.fast.ai/models/wt103/itos_wt103.pkl
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4161252 (4.0M) [text/plain]
Saving to: ‘assets/wikitext-103/itos_wt103.pkl’

itos_wt103.pkl      100%[===================>]   3.97M  7.05MB/s    in 0.6s    

2019-05-10 13:08:55 (7.05 MB/s) - ‘assets/wikitext-103/itos_wt103.pkl’ saved [4161252/4161252]



In [10]:
!wget -P assets/weights/ http://files.fast.ai/models/wt103/fwd_wt103.h5

--2019-05-10 13:08:57--  http://files.fast.ai/models/wt103/fwd_wt103.h5
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462387687 (441M) [text/plain]
Saving to: ‘assets/weights/fwd_wt103.h5’

fwd_wt103.h5        100%[===================>] 440.97M  41.0MB/s    in 11s     

2019-05-10 13:09:08 (39.2 MB/s) - ‘assets/weights/fwd_wt103.h5’ saved [462387687/462387687]



In [0]:
#!wget -xP assets/weights/  http://files.fast.ai/models/wt103/fwd_wt103_enc.h5
#!wget -xP assets/weights/  http://files.fast.ai/models/wt103/bwd_wt103_enc.h5
#!wget -xP assets/weights/  http://files.fast.ai/models/wt103/bwd_wt103.h5

## Install packages

In [12]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
     |████████████████████████████████| 576.2MB 24kB/s 


In [13]:
!pip install attrdict

In [0]:
import numpy as np
import csv
import urllib3
import os
import pathlib
import yaml
from attrdict import AttrDict
import pandas as pd

from fastai.text import Tokenizer
from fastai.text import partition_by_cores

CONFIG_PATH = str(pathlib.Path(os.getcwd()) / 'configs' / 'config.yaml')


# Alex Martelli's 'Borg'
# http://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html
class _Borg:
    _shared_state = {}

    def __init__(self):
        self.__dict__ = self._shared_state


class LoadParameters(_Borg):
    def __init__(self, fallback_file=CONFIG_PATH):
        _Borg.__init__(self)

        self.fallback_file = fallback_file
        self.params = self._read_yaml()

    def _read_yaml(self):
        with open(self.fallback_file) as f:
            config = yaml.load(f)
        return AttrDict(config)



def preprocess_imdb_sentiments(root_directory):
    """
    Transforms the IMDB sentiment dataset into a dataframe.

    IMBD dataset can be found on http://ai.stanford.edu/~amaas/data/sentiment/
    :param root_directory: Root directory containing the train and test folders
    :return:
    """

    classes = ['neg', 'pos']

    def get_texts(path):
        texts,labels = [],[]
        for idx, label in enumerate(classes):
            for fname in os.listdir(os.path.join(path, label)):
                if fname.endswith('.txt'):
                    with open(os.path.join(os.path.join(path, label), fname), 'r') as text:
                        texts.append(text.read())
                        labels.append(idx)

        texts = Tokenizer.proc_all_mp(partition_by_cores(texts))
        texts = [' '.join(text) + '<eos>' for text in texts]

        return np.array(texts), np.array(labels)

    trn_texts, trn_labels = get_texts(os.path.join(root_directory, 'train'))
    val_texts, val_labels = get_texts(os.path.join(root_directory, 'test'))

    col_names = ['labels', 'text']

    np.random.seed(42)
    trn_idx = np.random.permutation(len(trn_texts))
    val_idx = np.random.permutation(len(val_texts))

    trn_texts = trn_texts[trn_idx]
    val_texts = val_texts[val_idx]

    trn_labels = trn_labels[trn_idx]
    val_labels = val_labels[val_idx]

    df_trn = pd.DataFrame({'text':trn_texts, 'labels':trn_labels}, columns=col_names)
    df_val = pd.DataFrame({'text':val_texts, 'labels':val_labels}, columns=col_names)

    df_trn[df_trn['labels'] != 2].to_csv(os.path.join(root_directory, 'train.csv'), header=False, index=False)
    df_val.to_csv(os.path.join(root_directory, 'test.csv'), header=False, index=False)


def maybe_download(filename, source_url, work_directory):
    """Download the filename from the source url into the working directory if it does not exist."""
    if not os.path.exists(work_directory):
        os.makedirs(work_directory)
    filepath = os.path.join(work_directory, filename)
    if not os.path.exists(filepath):
        urllib3.disable_warnings()
        with urllib3.PoolManager() as http:
            r = http.request('GET', source_url)
            with open(filepath, 'wb') as fout:
                fout.write(r.read())

        print('{} succesfully downloaded'.format(filename))
    return filepath


def write_file(file_path, text_path, num_tokens):
    total_num_tokens = 0
    print(f'Writing to {file_path}...')
    with open(file_path, 'w', encoding='utf-8') as f_out:
        writer = csv.writer(f_out)
        with open(text_path, 'r') as f:
            for i, text in enumerate(f):

                writer.writerow([text])
                # f_out.write(text)

                # calculate approximate length based on tokens
                total_num_tokens += len(text.split())
                if total_num_tokens > num_tokens:
                    break
                if i % 10000 == 0:
                    print('Processed {:,} documents. Total # tokens: {:,}.'.format(i, total_num_tokens))
    print('{}. # documents: {:,}. # tokens: {:,}.'.format(file_path, i, total_num_tokens))


# Prepare Wikitxt

In [15]:
!ls assets/wikitext-103/

itos_wt103.pkl	test.txt  train.txt  valid.txt


In [0]:
import os
import pickle
from collections import Counter

from tqdm import tqdm
import re
import html

from fastai.text import partition_by_cores, Tokenizer
import spacy
import sys


class Missingdict(dict):
    """
    Own implementation of defaultdict, which does not add a new key: value pair for keys which were not in the dictionary
    before lookup.
    """
    def __init__(self, *args,  default_value=None, **kwargs,):
        super(Missingdict, self).__init__(*args, **kwargs)
        self.default_value = default_value

    def __missing__(self, key):
        return self.default_value


class Corpus(object):
    """
    Loads the train.txt, valid.txt and test.txt files and tokenizes them.
    The class then contains the train, valid and test corpus (with word2idx applied), as well as the word2idx dictionary.
    """

    def __init__(self, path, lang='en', max_vocab=30000, min_freq=10, word2idx=None):
        try:
            spacy.load(lang)
        except OSError:
            print(f'spacy tokenization model is not installed for {lang}.')
            lang = lang if lang in ['en', 'de', 'es', 'pt', 'fr', 'it', 'nl'] else 'xx'
            print(f'Command: python -m spacy download {lang}')
            sys.exit(1)

        self.max_vocab = max_vocab
        self.min_freq = min_freq
        self.lang = lang
        self.re1 = re.compile(r'  +')

        self.train_txt_path = os.path.join(path, 'train.txt')  # train.txt has 1 801 349 lines
        self.valid_txt_path = os.path.join(path, 'valid.txt')
        self.test_txt_path = os.path.join(path, 'test.txt')

        tokenized_train = self.tokenize(self.train_txt_path)
        if not word2idx:
            # Create word2idx from training corpus
            self.word2idx = self.create_word2idx(tokenized_train)
        else:
            self.word2idx = word2idx
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.train = [self.word2idx[word] for word in tokenized_train]
        del tokenized_train

        tokenized_valid = self.tokenize(self.valid_txt_path)
        self.valid = [self.word2idx[word] for word in tokenized_valid]
        del tokenized_valid

        tokenized_test = self.tokenize(self.test_txt_path)
        self.test = [self.word2idx[word] for word in tokenized_test]
        del tokenized_test

        self.word2idx = dict(word2idx)

    def fixup(self, x):
        """
        Fixes some observed weird tokens
        :param x:
        :return:
        """
        x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
            'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
            '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
            ' @-@ ','-').replace('\\', ' \\ ')
        return self.re1.sub(' ', html.unescape(x))

    def create_word2idx(self, tokenized_corpus):
        freq = Counter(tokenized_corpus)
        print(freq.most_common(25))

        idx2word = [o for o, c in freq.most_common(self.max_vocab) if c > self.min_freq]
        idx2word.insert(0, '_pad_')
        idx2word.insert(0, '_unk_')

        word2idx = {v: k for k, v in enumerate(idx2word)}
        word2idx = Missingdict(word2idx, default_value=0)

        return word2idx

    def tokenize(self, path):
        """
        Tokenizes a text file. Every 50000 lines of a file, the text will be tokenized with
        fast.ai's tokenizer.
        :param path:
        :param num_tokens:
        :return:
        """
        tokenized_corpus = []
        text = ''
        with open(path, 'r') as f:
            for i, line in tqdm(enumerate(f)):
                line = self.fixup(line)
                text += 'BOS' + line + 'EOS'
                if i % 50000 == 0 and i > 0:
                    tokenized_corpus += self._apply_tokenizer(text)
                    text = ''
        tokenized_corpus += self._apply_tokenizer(text)

        return tokenized_corpus

    def _apply_tokenizer(self, text):
        # Tokenizer creates a list, containing a list of each word pairs, such as
        #tokens = [[' ', 't_up', 'bos'], ['senjō'], ['no'], [' ', 't_up', 'eos']]
        tokens = Tokenizer(lang=self.lang).process_all(partition_by_cores(text.split(), n_cpus=8))
        # Flatten nested list
        return [word for sublist in tokens for word in sublist]


def create_corpus(PYTORCH_IDX2WORD_FILEPATH, WIKI103_FOLDER):
    # 1. Load words from pretrained pytorch language model
    # The itos_wt103.pkl file can be found on http://files.fast.ai/models/wt103/
    with open(PYTORCH_IDX2WORD_FILEPATH, 'rb') as f:
        words = pickle.load(f)
    word2idx = {word: idx for idx, word in enumerate(words)}
    word2idx = Missingdict(word2idx, default_value=0)

    # 2. Create corpus from Wiki103 files
    corpus = Corpus(WIKI103_FOLDER, word2idx=word2idx)

    to_save = [corpus.train, corpus.valid, corpus.test, corpus.word2idx, corpus.idx2word]
    with open(os.path.join(WIKI103_FOLDER, 'wikitext-103.corpus'), 'wb') as f:
        pickle.dump(to_save, f)



In [0]:


params = LoadParameters()
WIKI103_FOLDER = params.params['wiki103_text_folder']
PYTORCH_IDX2WORD_FILEPATH = params.params['pytorch_idx2word_filepath']
from utils.utils import LoadParameters

create_corpus(PYTORCH_IDX2WORD_FILEPATH, WIKI103_FOLDER)

1346349it [58:13, 525.29it/s]

# Train generic LM

In [0]:
import numpy as np


class BatchGenerator:

    def __init__(self, tokenized_text, batch_size, seq_len, modify_seq_len=True, only_last=False):
        """
        :param array tokenized_text: array of encoded text
        :param batch_size:
        :param bool only_last: Names the model, for which we need batches
        :param bool modify_seq_len: Determines, whether the sequence length should be randomly changed after each batch
        """
        self.tokenized_text = tokenized_text
        self.batch_size = batch_size
        self.modify_seq_len = modify_seq_len
        self.only_last = only_last
        self.seq_len = seq_len

        self.pos = 0

    @staticmethod
    def _random_modify_seq_len(seq_len):
        seq_len = seq_len if np.random.random() < 0.95 else seq_len / 2.
        # Prevent excessively small or negative sequence lengths
        return max(5, int(np.random.normal(seq_len, 5)))


    def get_sample(self, pos, batch_seq_len):
        """
        Returns one x, y pair
        :param pos:
        :param batch_seq_len:
        :return:
        """
        start = (pos + 1) % len(self.tokenized_text)
        end = (pos + 1 + batch_seq_len) % len(self.tokenized_text)
        if start > end:  # text sequence is exhausted -> roll over to the start again
            start = len(self.tokenized_text) - batch_seq_len
            end = len(self.tokenized_text)
           
        data = self.tokenized_text[start -1: end - 1]
        target = self.tokenized_text[end - 1] if self.only_last else self.tokenized_text[start:end]
        # need to expand the dimension, such that Y has shape (batch_size, seq_len, 1) (for normal model),
        # or (batch_size, 1) (for many_to_one model)
        # -> needed for sparse_categorical_crossentropy loss
        target = np.expand_dims(target, axis=1)

        return data, target

    def generate_one_batch(self, pos, batch_seq_len):
        """
        Batch is of size:
        X: (batch_size, seq_len) Y: (batch_size, seq_len, 1) for self.model_description == normal
        X: (batch_size, seq_len) Y: (batch_size, 1) for self.model_description == many_to_one
        X: (2, batch_size, seq_len) Y: (batch_size, seq_len) for self.model_description == fast
        :param int pos:
        :param int seq_len:
        :return:
        """
        X = []
        Y = []
        for i in range(self.batch_size):
            x, y = self.get_sample(pos, batch_seq_len)
            X.append(x)
            Y.append(y)
            pos = (pos + batch_seq_len) % len(self.tokenized_text)

        return np.array(X), np.array(Y)

    def __iter__(self):
        return self

    def __next__(self):
        """
        Generates a batch for training/validation
        :param seq_len:
        :return:
        """
        batch_seq_len = self.seq_len
        if self.modify_seq_len:
            batch_seq_len = self._random_modify_seq_len(batch_seq_len)

        X, Y = self.generate_one_batch(self.pos, batch_seq_len)
        self.pos = (self.pos + batch_seq_len * self.batch_size) % len(self.tokenized_text)

        return X, Y


In [0]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"
from keras.layers import Input, CuDNNLSTM, Embedding, Dense, LSTM, TimeDistributed, Dropout
from keras.models import Model

# -*- coding: utf-8 -*-
# from https://raw.githubusercontent.com/DingKe/nn_playground/master/qrnn/qrnn.py


from __future__ import absolute_import
import numpy as np

from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.layers import Layer, InputSpec
from keras.utils.conv_utils import conv_output_length


def _dropout(x, level, noise_shape=None, seed=None):
    x = K.dropout(x, level, noise_shape, seed)
    x *= (1. - level)  # compensate for the scaling by the dropout
    return x


class QRNN(Layer):
    '''Quasi RNN

    # Arguments
        units: dimension of the internal projections and the final output.

    # References
        - [Quasi-recurrent Neural Networks](http://arxiv.org/abs/1611.01576)
    '''

    def __init__(self, units, window_size=2, stride=1,
                 return_sequences=False, go_backwards=False,
                 stateful=False, unroll=False, activation='tanh',
                 kernel_initializer='uniform', bias_initializer='zero',
                 kernel_regularizer=None, bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None, bias_constraint=None,
                 dropout=0, use_bias=True, input_dim=None, input_length=None,
                 **kwargs):
        self.return_sequences = return_sequences
        self.go_backwards = go_backwards
        self.stateful = stateful
        self.unroll = unroll

        self.units = units
        self.window_size = window_size
        self.strides = (stride, 1)

        self.use_bias = use_bias
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = dropout
        self.supports_masking = True
        self.input_spec = [InputSpec(ndim=3)]
        self.input_dim = input_dim
        self.input_length = input_length
        if self.input_dim:
            kwargs['input_shape'] = (self.input_length, self.input_dim)
        super(QRNN, self).__init__(**kwargs)

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        self.input_dim = input_shape[2]
        self.input_spec = InputSpec(shape=(batch_size, None, self.input_dim))
        self.state_spec = InputSpec(shape=(batch_size, self.units))

        self.states = [None]
        if self.stateful:
            self.reset_states()

        kernel_shape = (self.window_size, 1, self.input_dim, self.units * 3)
        self.kernel = self.add_weight(name='kernel',
                                      shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(name='bias',
                                        shape=(self.units * 3,),
                                        initializer=self.bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)

        self.built = True

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        length = input_shape[1]
        if length:
            length = conv_output_length(length + self.window_size - 1,
                                        self.window_size, 'valid',
                                        self.strides[0])
        if self.return_sequences:
            return (input_shape[0], length, self.units)
        else:
            return (input_shape[0], self.units)

    def compute_mask(self, inputs, mask):
        if self.return_sequences:
            return mask
        else:
            return None

    def get_initial_states(self, inputs):
        # build an all-zero tensor of shape (samples, units)
        initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
        initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
        initial_state = K.expand_dims(initial_state)  # (samples, 1)
        initial_state = K.tile(initial_state, [1, self.units])  # (samples, units)
        initial_states = [initial_state for _ in range(len(self.states))]
        return initial_states

    def reset_states(self, states=None):
        if not self.stateful:
            raise AttributeError('Layer must be stateful.')
        if not self.input_spec:
            raise RuntimeError('Layer has never been called '
                               'and thus has no states.')

        batch_size = self.input_spec.shape[0]
        if not batch_size:
            raise ValueError('If a QRNN is stateful, it needs to know '
                             'its batch size. Specify the batch size '
                             'of your input tensors: \n'
                             '- If using a Sequential model, '
                             'specify the batch size by passing '
                             'a `batch_input_shape` '
                             'argument to your first layer.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a '
                             '`batch_shape` argument to your Input layer.')

        if self.states[0] is None:
            self.states = [K.zeros((batch_size, self.units))
                           for _ in self.states]
        elif states is None:
            for state in self.states:
                K.set_value(state, np.zeros((batch_size, self.units)))
        else:
            if not isinstance(states, (list, tuple)):
                states = [states]
            if len(states) != len(self.states):
                raise ValueError('Layer ' + self.name + ' expects ' +
                                 str(len(self.states)) + ' states, '
                                                         'but it received ' + str(len(states)) +
                                 'state values. Input received: ' +
                                 str(states))
            for index, (value, state) in enumerate(zip(states, self.states)):
                if value.shape != (batch_size, self.units):
                    raise ValueError('State ' + str(index) +
                                     ' is incompatible with layer ' +
                                     self.name + ': expected shape=' +
                                     str((batch_size, self.units)) +
                                     ', found shape=' + str(value.shape))
                K.set_value(state, value)

    def __call__(self, inputs, initial_state=None, **kwargs):
        # If `initial_state` is specified,
        # and if it a Keras tensor,
        # then add it to the inputs and temporarily
        # modify the input spec to include the state.
        if initial_state is not None:
            if hasattr(initial_state, '_keras_history'):
                # Compute the full input spec, including state
                input_spec = self.input_spec
                state_spec = self.state_spec
                if not isinstance(state_spec, list):
                    state_spec = [state_spec]
                self.input_spec = [input_spec] + state_spec

                # Compute the full inputs, including state
                if not isinstance(initial_state, (list, tuple)):
                    initial_state = [initial_state]
                inputs = [inputs] + list(initial_state)

                # Perform the call
                output = super(QRNN, self).__call__(inputs, **kwargs)

                # Restore original input spec
                self.input_spec = input_spec
                return output
            else:
                kwargs['initial_state'] = initial_state
        return super(QRNN, self).__call__(inputs, **kwargs)

    def call(self, inputs, mask=None, initial_state=None, training=None):
        # input shape: `(samples, time (padded with zeros), input_dim)`
        # note that the .build() method of subclasses MUST define
        # self.input_spec and self.state_spec with complete input shapes.
        if isinstance(inputs, list):
            initial_states = inputs[1:]
            inputs = inputs[0]
        elif initial_state is not None:
            pass
        elif self.stateful:
            initial_states = self.states
        else:
            initial_states = self.get_initial_states(inputs)

        if len(initial_states) != len(self.states):
            raise ValueError('Layer has ' + str(len(self.states)) +
                             ' states but was passed ' +
                             str(len(initial_states)) +
                             ' initial states.')
        input_shape = K.int_shape(inputs)
        if self.unroll and input_shape[1] is None:
            raise ValueError('Cannot unroll a RNN if the '
                             'time dimension is undefined. \n'
                             '- If using a Sequential model, '
                             'specify the time dimension by passing '
                             'an `input_shape` or `batch_input_shape` '
                             'argument to your first layer. If your '
                             'first layer is an Embedding, you can '
                             'also use the `input_length` argument.\n'
                             '- If using the functional API, specify '
                             'the time dimension by passing a `shape` '
                             'or `batch_shape` argument to your Input layer.')
        constants = self.get_constants(inputs, training=None)
        preprocessed_input = self.preprocess_input(inputs, training=None)

        last_output, outputs, states = K.rnn(self.step, preprocessed_input,
                                             initial_states,
                                             go_backwards=self.go_backwards,
                                             mask=mask,
                                             constants=constants,
                                             unroll=self.unroll,
                                             input_length=input_shape[1])
        if self.stateful:
            updates = []
            for i in range(len(states)):
                updates.append((self.states[i], states[i]))
            self.add_update(updates, inputs)

        # Properly set learning phase
        if 0 < self.dropout < 1:
            last_output._uses_learning_phase = True
            outputs._uses_learning_phase = True

        if self.return_sequences:
            return outputs
        else:
            return last_output

    def preprocess_input(self, inputs, training=None):
        if self.window_size > 1:
            inputs = K.temporal_padding(inputs, (self.window_size - 1, 0))
        inputs = K.expand_dims(inputs, 2)  # add a dummy dimension

        output = K.conv2d(inputs, self.kernel, strides=self.strides,
                          padding='valid',
                          data_format='channels_last')
        output = K.squeeze(output, 2)  # remove the dummy dimension
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')

        if self.dropout is not None and 0. < self.dropout < 1.:
            z = output[:, :, :self.units]
            f = output[:, :, self.units:2 * self.units]
            o = output[:, :, 2 * self.units:]
            f = K.in_train_phase(1 - _dropout(1 - f, self.dropout), f, training=training)
            return K.concatenate([z, f, o], -1)
        else:
            return output

    def step(self, inputs, states):
        prev_output = states[0]

        z = inputs[:, :self.units]
        f = inputs[:, self.units:2 * self.units]
        o = inputs[:, 2 * self.units:]

        z = self.activation(z)
        f = f if self.dropout is not None and 0. < self.dropout < 1. else K.sigmoid(f)
        o = K.sigmoid(o)

        output = f * prev_output + (1 - f) * z
        output = o * output

        return output, [output]

    def get_constants(self, inputs, training=None):
        return []

    def get_config(self):
        config = {'units': self.units,
                  'window_size': self.window_size,
                  'stride': self.strides[0],
                  'return_sequences': self.return_sequences,
                  'go_backwards': self.go_backwards,
                  'stateful': self.stateful,
                  'unroll': self.unroll,
                  'use_bias': self.use_bias,
                  'dropout': self.dropout,
                  'activation': activations.serialize(self.activation),
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'input_dim': self.input_dim,
                  'input_length': self.input_length}
        base_config = super(QRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class TiedEmbeddingsTransposed(Layer):
    """Layer for tying embeddings in an output layer.
    A regular embedding layer has the shape: V x H (V: size of the vocabulary. H: size of the projected space).
    In this layer, we'll go: H x V.
    With the same weights than the regular embedding.
    In addition, it may have an activation.
    # References
        - [ Using the Output Embedding to Improve Language Models](https://arxiv.org/abs/1608.05859)
    """

    def __init__(self, tied_to=None,
                 activation=None,
                 **kwargs):
        super(TiedEmbeddingsTransposed, self).__init__(**kwargs)
        self.tied_to = tied_to
        self.activation = activations.get(activation)

    def build(self, input_shape):
        self.transposed_weights = K.transpose(self.tied_to.weights[0])
        self.built = True

    def compute_mask(self, inputs, mask=None):
        return mask

    def compute_output_shape(self, input_shape):
        return input_shape[0], K.int_shape(self.tied_to.weights[0])[0]

    def call(self, inputs, mask=None):
        output = K.dot(inputs, self.transposed_weights)
        if self.activation is not None:
            output = self.activation(output)
        return output


    def get_config(self):
        config = {'activation': activations.serialize(self.activation)
                  }
        base_config = super(TiedEmbeddingsTransposed, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))




def build_language_model(num_words, embedding_size=400, rnn_sizes=(1150, 1150),
                         dropout=0.1, dropouth=0.3, dropouti=0.2, dropoute=0.1, wdrop=0.5,
                         tie_weights=True, use_qrnn=False, use_gpu=True, only_last=False):

    inp = Input(shape=(None,), name='input')
    emb = Embedding(num_words, embedding_size, name='embedding')
    emb_inp = emb(inp)
    emb_inp = Dropout(dropouti)(emb_inp)

    if use_qrnn:
        rnn = QRNN(rnn_sizes[0], return_sequences=True, window_size=2)(emb_inp)
        for rnn_size in rnn_sizes[1:]:
            rnn = QRNN(rnn_size, return_sequences=True, window_size=1)(rnn)
        if only_last:
            rnn = QRNN(embedding_size, return_sequences=False, window_size=1, name='final_rnn_layer')(rnn)
        else:
            rnn = QRNN(embedding_size, return_sequences=True, window_size=1, name='final_rnn_layer')(rnn)
    else:
        RnnUnit = CuDNNLSTM if use_gpu else LSTM
        rnn = RnnUnit(rnn_sizes[0], return_sequences=True, name='0_rnn_layer')(emb_inp)
        for i, rnn_size in enumerate(rnn_sizes[1:]):
            rnn = RnnUnit(rnn_size, return_sequences=True, name='{}_rnn_layer'.format(i + 1))(rnn)
        if only_last:
            rnn = RnnUnit(embedding_size, return_sequences=False, name='final_rnn_layer')(rnn)
        else:
            rnn = RnnUnit(embedding_size, return_sequences=True, name='final_rnn_layer')(rnn)

    if tie_weights:
        softmax_layer = TiedEmbeddingsTransposed(tied_to=emb, activation='softmax')
    else:
        softmax_layer = Dense(num_words, activation='softmax')

    if only_last:
        logits = softmax_layer(rnn)
    else:
        logits = TimeDistributed(softmax_layer)(rnn)

    out = Dropout(dropout)(logits)
    model = Model(inputs=inp, outputs=out)
    return model


In [0]:
import pickle

import keras.backend as K
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam



def evaluate_model(model, word2idx, test_sentence, num_predictions=5):
    """
    Visual preidictions of the language model. The test_sentence is appended with num_predictions words,
    which are predicted as the next words from the model.
    :param str test_sentence:
    :param int num_predictions:
    :return: None
    """

    idx2word = {i: w for w, i in word2idx.items()}
    test_sentence = test_sentence.split()
    encoded_sentence = [word2idx[w] for w in test_sentence]

    for i in range(num_predictions):
        X = np.reshape(encoded_sentence, (1, len(encoded_sentence)))

        pred = model.predict(X)
        answer = np.argmax(pred, axis=2)

        predicted_idx = answer[0][-2]
        encoded_sentence.append(predicted_idx)

    print(' '.join([idx2word[i] for i in encoded_sentence]))




# 1. Load parameters from config.yaml file
params = LoadParameters()
WIKIPEDIA_CORPUS_FILE = params.params['wikipedia_corpus_file']
LANGUAGE_MODEL_WEIGHT = params.params['language_model_weight']
LANGUAGE_MODEL_PARAMS = params.params['lm_params']

epochs = params.params['lm_epochs']
batch_size = params.params['lm_batch_size']
valid_batch_size = params.params['lm_valid_batch_size']
seq_len = params.params['lm_seq_len']

# 2. Load Corpus
[train, valid, test, word2idx, idx2word] = pickle.load(open(WIKIPEDIA_CORPUS_FILE, 'rb'))

train_gen = BatchGenerator(train, batch_size, seq_len, modify_seq_len=True)
valid_gen = BatchGenerator(valid, valid_batch_size, seq_len, modify_seq_len=True)

K.clear_session()
num_words = len(word2idx) + 1

model = build_language_model(num_words, **LANGUAGE_MODEL_PARAMS)
model.compile(loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'],
              optimizer=Adam(lr=3e-4, beta_1=0.8, beta_2=0.99))

model.summary()

callbacks = [EarlyStopping(patience=5),
             ModelCheckpoint(LANGUAGE_MODEL_WEIGHT, save_weights_only=True)]

history = model.fit_generator(train_gen,
                              steps_per_epoch=len(train) // (seq_len * batch_size),
                              epochs=epochs,
                              validation_data=valid_gen,
                              validation_steps=len(valid) // (seq_len * batch_size),
                              callbacks=callbacks,
                              )

evaluate_model(model, word2idx, 'i feel sick and go to the ')


# Fine tune language model

In [0]:
"""
Implementation of Discriminative fine-tuning as explained in https://arxiv.org/pdf/1801.06146.pdf

Adatped from https://github.com/brunoklein99/srcnn/blob/5e874eb161d4d27cfdb6ac9b2196b3ad154fc672/LRMultiplierSGD.py#L46
"""

import keras.backend as K
from keras.legacy import interfaces
from keras.optimizers import Optimizer


class LRMultiplierSGD(Optimizer):
    """Stochastic gradient descent optimizer.
    Implements the Discriminative fine-tuning as explained in https://arxiv.org/pdf/1801.06146.pdf
    Current implementation is such that it is assumed that each layer consists of two update parameters,
    namely weights and bias.

    Includes support for momentum,
    learning rate decay, and Nesterov momentum.
    Example use:
    opt = LRMultiplierSGD(lr=1e-4, momentum=0.9, multipliers=[1, 1, 1, 1, 0.1, 0.1])
    model.compile(optimizer=opt, loss=mean_squared_error, metrics=[psnr])

    # Arguments
        lr: float >= 0. Learning rate.
        momentum: float >= 0. Parameter that accelerates SGD
            in the relevant direction and dampens oscillations.
        decay: float >= 0. Learning rate decay over each update.
        nesterov: boolean. Whether to apply Nesterov momentum.
    """

    def __init__(self, lr=0.01, momentum=0., decay=0., discriminative_decay=1 / 2.6,
                 nesterov=False, **kwargs):
        super(LRMultiplierSGD, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.momentum = K.variable(momentum, name='momentum')
            self.decay = K.variable(decay, name='decay')
            self.discriminative_decay = discriminative_decay
        self.initial_decay = decay
        self.nesterov = nesterov


    def _set_up_discriminative_fine_tuning(self, params):
        """
        Sets up the decay for different layers in the network. The decay is calculated according to
        decay[layer_(i + 1)] = decay[layer_i] * self.discriminative_decay. The layer last to the output is layer_0
        and is set such that decay[layer_0] = 1.
        :param params:
        :return:
        """
        # Example of param.name:
        # lstm_1/kernel:0
        # lstm_1/recurrent_kernel:0
        # lstm_1/bias:0
        # lstm_2/kernel:0
        # lstm_2/recurrent_kernel:0
        # lstm_2/bias:0
        # dense_1/kernel:0
        # dense_1/bias:0
        names = [param.name.split('/')[0] for param in params]
        number_of_layers = len(set(names))

        def list_to_depth(names):
            """
            ['lstm_1', 'lstm_1', 'lstm_1', 'lstm_2', 'lstm_2', 'lstm_2', 'dense_1', 'dense_1']
            goes to
            [1, 1, 1, 2, 2, 2, 3, 3]
            :param names:
            :return:
            """
            # TODO: refactor
            layer_depths = [1]
            layer_depth = 1
            for i, name in enumerate(names[1:]):
                if name != names[i]:
                    layer_depth += 1
                layer_depths.append(layer_depth)
            return layer_depths

        layer_depths = list_to_depth(names)

        layer_decay = [K.variable(self.discriminative_decay ** (number_of_layers - depth)) for depth in layer_depths]
        return layer_decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        """

        :param loss:
        :param list[tf.Variable] params: list of tensorflow weights and biases
        :return:
        """
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))
        # momentum
        shapes = [K.int_shape(p) for p in params]
        moments = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + moments

        # Discriminative fine-tuning:
        # each layer is updated according to the idea from https://arxiv.org/pdf/1801.06146.pdf, after eq(2)

        # The learning rate of the uppermost trainable layer is decreased by discriminative_fine_tuning
        # len(params) // 2 is (approximately) the number of layers on the model -> weight+ bias per each layer
        discriminative_fine_tuning = self._set_up_discriminative_fine_tuning(params)

        for i, (p, g, m) in enumerate(zip(params, grads, moments)):

            v = self.momentum * m - (lr * discriminative_fine_tuning[i]) * g  # velocity

            self.updates.append(K.update(m, v))

            if self.nesterov:
                new_p = p + self.momentum * v - (lr * discriminative_fine_tuning[i]) * g
            else:
                new_p = p + v

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'momentum': float(K.get_value(self.momentum)),
                  'decay': float(K.get_value(self.decay)),
                  'nesterov': self.nesterov}
        base_config = super(LRMultiplierSGD, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
"""
Fintunes a pretrained language model on a new language corpus.
"""

import os
import pickle

import keras.backend as K
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import sparse_categorical_crossentropy


def read_df(df):
    """
    Maps all entries in df.text into one single list.
    :param df:
    :return:
    """
    texts = [text for text in df.text]
    texts = ' '.join(texts)
    return texts


def update_word2idx(word_list, word2idx):
    '''
    Updates the word2idx dictionary with all words which are in the word_list, but do not appear
    in the word2idx.keys()
    '''
    words_not_in_corpus = list(set(word_list) - set(word2idx.keys()))
    additional_word2idx = {words_not_in_corpus[i]: len(word2idx) + i
                           for i in range(len(words_not_in_corpus))
                           }
    word2idx.update(additional_word2idx)

    return word2idx


def update_embedding_weights(embedding_weights, num_words_not_in_corpus):
    embedding_weights = embedding_weights[0]  # shape (len(language_corpus.word2idx), embedding_size)
    mean_embedding_vector = embedding_weights.mean(axis=0)
    embedding_weights = np.append(embedding_weights, [mean_embedding_vector for _ in range(num_words_not_in_corpus)], axis=0)
    return embedding_weights


def update_language_model(language_model, num_words_not_in_corpus, **kwargs):
    weights = language_model.get_weights()

    old_embedding_weights = language_model.get_layer('embedding').get_weights()  # shape (len(language_corpus.word2idx), embedding_size)
    new_embedding_weights = update_embedding_weights(old_embedding_weights, num_words_not_in_corpus)

    weights[0] = new_embedding_weights

    updated_language_model = build_language_model(num_words=new_embedding_weights.shape[0],
                                                  **kwargs)

    updated_language_model.set_weights(weights)

    return updated_language_model


import pandas as pd
from collections import Counter, defaultdict



# 1. Load parameters from config.yaml
params = LoadParameters()

PYTORCH_ITOS_FILEPATH = params.params['pytorch_idx2word_filepath']
WEIGTHS_FILEPATH = params.params['language_model_weight']
FINETUNED_WEIGTHS_FILEPATH = params.params['finetuned_language_model_weight']
FINETUNED_WORD2IDX_FILEPATH = params.params['finetuned_word2idx_filepath']
LANGUAGE_MODEL_PARAMS = params.params['lm_params']
FINETUNED_CORPUS_FILEPATH = params.params['finetuned_corpus_filepath']

batch_size = params.params['lm_batch_size']
seq_length = params.params['lm_seq_len']

# 2. Initialize pretrained language model.
K.clear_session()
with open(PYTORCH_ITOS_FILEPATH, 'rb') as f:
    words = pickle.load(f)

word2idx = {word: idx for idx, word in enumerate(words)}

word2idx = defaultdict(lambda: word2idx['_unk_'], word2idx)

num_words = len(word2idx)

language_model = build_language_model(num_words, **LANGUAGE_MODEL_PARAMS)
language_model.summary()
language_model.load_weights(WEIGTHS_FILEPATH)

# 3. Open target training dataset. We assume that the dataframes contains the already tokenized sentences.
train_df = pd.read_csv(os.path.join(FINETUNED_CORPUS_FILEPATH, 'train.csv'), names=['mood', 'text'])
valid_df = pd.read_csv(os.path.join(FINETUNED_CORPUS_FILEPATH, 'test.csv'), names=['mood', 'text'])

train_text = read_df(train_df)
valid_text = read_df(valid_df)

text = train_text + valid_text

unique_words = [o for o, c in Counter(text).most_common(100000) if c > 10]

# 4. Add new words to the word2idx dictionary and update the language model.
num_words_not_in_corpus = len(set(unique_words) - set(word2idx.keys()))
word2idx = update_word2idx(unique_words, word2idx)
language_model = update_language_model(language_model, num_words_not_in_corpus, **LANGUAGE_MODEL_PARAMS)

language_model.summary()

# 5. Prepare training and validation data
train = [word2idx[word] for word in train_text]
valid = [word2idx[word] for word in valid_text]

# 6. Finetune model
train_gen = BatchGenerator(train, batch_size, seq_length, modify_seq_len=True)
valid_gen = BatchGenerator(valid, batch_size, seq_length, modify_seq_len=True)

callbacks = [EarlyStopping(patience=5),
             ModelCheckpoint(FINETUNED_WEIGTHS_FILEPATH, save_weights_only=True)]

language_model.compile(loss=sparse_categorical_crossentropy,
                       metrics=['sparse_categorical_accuracy'],
                       optimizer=LRMultiplierSGD(lr=0.2 * 3e-4, momentum=0., decay=0., nesterov=False)
                       )

language_model.fit_generator(train_gen,
                             steps_per_epoch=len(train) // (seq_length * batch_size),
                             epochs=20,
                             validation_data=valid_gen,
                             validation_steps=len(valid) // (seq_length * batch_size),
                             callbacks=callbacks,
                             )

evaluate_model(language_model, word2idx, 'i feel sick and go to the', num_predictions=5)

# 7. Save word2idx dictionary
with open(FINETUNED_WORD2IDX_FILEPATH, 'wb') as f:
    pickle.dump(word2idx, f)


# ULMFiT integration (LM+Cls)

In [0]:
from keras.layers import Lambda, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, BatchNormalization, Dense, Dropout
from keras.models import Model


def build_classification_model(language_model, num_labels, dense_units=(128, 128), dropouts=(0.1, 0.1)):
    """
    Transfer model for language classification. Implementation of the transfer model as explained in
    https://arxiv.org/abs/1801.06146.
    :param language_model:
    :param lr:
    :param lr_d:
    :param kernel_size1:
    :param kernel_size2:
    :param dense_units:
    :param dropout:
    :param filters:
    :return:
    """
    rnn_output = language_model.get_layer('final_rnn_layer').output

    avg_pool = GlobalAveragePooling1D()(rnn_output)
    max_pool = GlobalMaxPooling1D()(rnn_output)
    last_rnn_output = Lambda(lambda x: x[:, -1, :])(rnn_output)

    x = concatenate([avg_pool, max_pool, last_rnn_output])

    for num_units, dropout in zip(dense_units, dropouts):
        x = Dense(num_units, activation='relu') (x)
        x = Dropout(dropout)(x)
        x = BatchNormalization()(x)

    x = Dense(num_labels, activation="sigmoid")(x)

    model = Model(inputs=language_model.input, outputs=x)

    return model

In [0]:
import pickle

import keras.backend as K
from keras.losses import categorical_crossentropy

def train_classifiaction_model(classification_model, X_train, y_train, epochs_list, learning_rates_list):
    """
    Implementation of transfer training by unfreezing the layers step by step.
    :param classification_model:
    :param X_train:
    :param y_train:
    :param List[int] epochs_list: list containing the number of epochs for each unfreezing step
    :param List[int] learning_rates_list: list containing the learning rates for each unfreezing step
    :return:
    """
    for layer in classification_model.layers:
        layer.trainable = False

    for i, layer in enumerate(reversed(classification_model.layers)):
        layer.trainable = True

        classification_model.compile(loss=categorical_crossentropy, optimizer=LRMultiplierSGD(lr=learning_rates_list[i],
                                                                                               momentum=0., decay=0.,
                                                                                               nesterov=False))

        classification_model.fit(X_train, y_train,
                                 batch_size=None,
                                 epochs=epochs_list[i],
                                 verbose=1,
                                 callbacks=None,
                                 validation_split=0.1,
                                 shuffle=True)

    return classification_model



In [0]:

# 1. Load parameters from config.yaml
params = LoadParameters()

CORPUS_FILEPATH = params.params['wikipedia_corpus_file']
TRAINING_DATA_FILEPATH = params.params['classification_csv']
CLASSIFICATON_MODEL_FILE = params.params['classifiaction_language_model_weight']

FINETUNED_WEIGTHS_FILEPATH = params.params['finetuned_language_model_weight']
NUMBER_OF_LABELS = params.params['number_of_labels']

# 2. Initialize pretrained language model.
K.clear_session()
wikitext_corpus = pickle.load(open(CORPUS_FILEPATH,'rb'))
num_words = len(wikitext_corpus.word2idx) +1

language_model = build_language_model(num_words, embedding_size=300, use_gpu=True)
language_model.summary()
language_model.load_weights(FINETUNED_WEIGTHS_FILEPATH)

# 3. Initialize classifiaction_model
classification_model = build_classification_model(language_model, NUMBER_OF_LABELS, **params.params['cm_params'])

# 4. Load X_train from pickle file
with open(TRAINING_DATA_FILEPATH, 'rb') as f:
    X_train, y_train = pickle.load(f)

# 5. Train classification model
epochs_list = [1 for layer in classification_model.layers]
learning_rates_list = [0.01 for layer in classification_model.layers]
train_classifiaction_model(classification_model, X_train, y_train, epochs_list, learning_rates_list)

# 6. Save classification model
classification_model.save(CLASSIFICATON_MODEL_FILE, overwrite=True)